<a href="https://colab.research.google.com/github/HarryPotter12/PractiseML/blob/master/20_newsgroup_kernel_kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [0]:
from sklearn.datasets import load_files
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from matplotlib.pyplot import cm
from matplotlib import pyplot as plt
import numpy as np
import time
from sklearn import metrics

In [0]:
newsgroups_full = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

In [0]:
print(newsgroups_full.data[:3])

["\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n", 'My brother is in the market for a high-performance video card that supports\nVESA local bus with 1-2MB RAM.  Does anyone have suggestions/ideas on:\n\n  - Diamond Stealth Pro Local Bus\n\n  - Orchid Farenheit 1280\n\n  - ATI Graphics Ultra Pro\n\n  - Any other high-perf

In [0]:
X = newsgroups_full.data
y = newsgroups_full.target

In [0]:
vectorizer = TfidfVectorizer(max_features = 10000)
tfidf = vectorizer.fit_transform(X)

In [0]:
tfidf.shape

(18846, 10000)

In [0]:
vectorizer_sub = TfidfVectorizer(sublinear_tf=True, max_features = 10000)
tfidf_sub = vectorizer.fit_transform(X)

In [0]:
tfidf_sub.shape

(18846, 10000)

In [0]:
vectorizer_bigram = TfidfVectorizer(ngram_range=(1, 2), max_features = 10000)
tfidf_bigram = vectorizer_bigram.fit_transform(X)

In [0]:
tfidf_bigram.shape

(18846, 10000)

In [0]:
vectorizer_max = TfidfVectorizer(norm='max', max_features = 10000)
tfidf_max = vectorizer_max.fit_transform(X)

In [0]:
tfidf_max.shape

(18846, 10000)

for i in range(18846):
  for j in range(10000):
    if tfidf.toarray()[i][j] != tfidf_max.toarray()[i][j]:
      print("i = %d j = %d tfidf = %f tfidf_max = %f No match" % (i, j, tfidf.toarray()[i][j], tfidf_max.toarray()[i][j]))

In [0]:
"""Kernel K-means"""

# Author: Mathieu Blondel <mathieu@mblondel.org>
# License: BSD 3 clause

import numpy as np

from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.utils import check_random_state


class KernelKMeans(BaseEstimator, ClusterMixin):
    """
    Kernel K-means
    
    Reference
    ---------
    Kernel k-means, Spectral Clustering and Normalized Cuts.
    Inderjit S. Dhillon, Yuqiang Guan, Brian Kulis.
    KDD 2004.
    """

    def __init__(self, n_clusters=3, init='k-means++', max_iter=50, tol=1e-3, random_state=None,
                 kernel="rbf", gamma=None, degree=3, coef0=1,
                 kernel_params=None, verbose=0):
        self.n_clusters = n_clusters
        self.init = init
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        self.kernel = kernel
        self.gamma = gamma
        self.degree = degree
        self.coef0 = coef0
        self.kernel_params = kernel_params
        self.verbose = verbose
        
    @property
    def _pairwise(self):
        return self.kernel == "precomputed"

    def _get_kernel(self, X, Y=None):
        if callable(self.kernel):
            params = self.kernel_params or {}
        else:
            params = {"gamma": self.gamma,
                      "degree": self.degree,
                      "coef0": self.coef0}
        return pairwise_kernels(X, Y, metric=self.kernel,
                                filter_params=True, **params)

    def fit(self, X, y=None, sample_weight=None):
        n_samples = X.shape[0]

        K = self._get_kernel(X)

        sw = sample_weight if sample_weight else np.ones(n_samples)
        self.sample_weight_ = sw

        rs = check_random_state(self.random_state)
        self.labels_ = rs.randint(self.n_clusters, size=n_samples)

        dist = np.zeros((n_samples, self.n_clusters))
        self.within_distances_ = np.zeros(self.n_clusters)

        for it in range(self.max_iter):
            dist.fill(0)
            self._compute_dist(K, dist, self.within_distances_,
                               update_within=True)
            labels_old = self.labels_
            self.labels_ = dist.argmin(axis=1)

            # Compute the number of samples whose cluster did not change 
            # since last iteration.
            n_same = np.sum((self.labels_ - labels_old) == 0)
            if 1 - float(n_same) / n_samples < self.tol:
                if self.verbose:
                    print("Converged at iteration", it + 1)
                break

        self.X_fit_ = X

        return self

    def _compute_dist(self, K, dist, within_distances, update_within):
        """Compute a n_samples x n_clusters distance matrix using the 
        kernel trick."""
        sw = self.sample_weight_

        for j in range(self.n_clusters):
            mask = self.labels_ == j

            if np.sum(mask) == 0:
                raise ValueError("Empty cluster found, try smaller n_cluster.")

            denom = sw[mask].sum()
            denomsq = denom * denom

            if update_within:
                KK = K[mask][:, mask]  # K[mask, mask] does not work.
                dist_j = np.sum(np.outer(sw[mask], sw[mask]) * KK / denomsq)
                within_distances[j] = dist_j
                dist[:, j] += dist_j
            else:
                dist[:, j] += within_distances[j]

            dist[:, j] -= 2 * np.sum(sw[mask] * K[:, mask], axis=1) / denom

    def predict(self, X):
        K = self._get_kernel(X, self.X_fit_)
        n_samples = X.shape[0]
        dist = np.zeros((n_samples, self.n_clusters))
        self._compute_dist(K, dist, self.within_distances_,
                           update_within=False)
        return dist.argmin(axis=1)

In [0]:
gammas = [0.01, 0.1, 1, 10, 100]

In [0]:
best_sil = float("-inf")
best_gamma = 1000
for my_gamma in gammas:
#  print("For gamma = ", my_gamma)
  km = KernelKMeans(n_clusters=20, max_iter=100, random_state=0, verbose=1, kernel="rbf", gamma=my_gamma)
  y_pred = km.fit_predict(tfidf)
#  print("Computing silhouette_score ")
  silhouette = metrics.silhouette_score(tfidf, y_pred)
  print("The silhouette_score for gamma = %f is = %.4f" % (my_gamma, silhouette))
  if silhouette > best_sil:
#    print("Old Sil : ", best_sil)
    best_sil = silhouette
    best_gamma = my_gamma
#    print("New Sil : ", best_sil, "gamma : ", best_gamma)

Converged at iteration 28
The silhouette_score for gamma = 0.010000 is = -0.0179
Converged at iteration 40
The silhouette_score for gamma = 0.100000 is = -0.2160
Converged at iteration 28
The silhouette_score for gamma = 1.000000 is = -0.2162
Converged at iteration 3
The silhouette_score for gamma = 10.000000 is = -0.2307
Converged at iteration 3
The silhouette_score for gamma = 100.000000 is = -0.2307


In [0]:
print("best_gamma : ", best_gamma)
km = KernelKMeans(n_clusters=20, max_iter=100, random_state=0, verbose=1, kernel="rbf", gamma=best_gamma)
y_pred = km.fit_predict(tfidf)

best_gamma :  0.01
Converged at iteration 28


In [0]:
print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(y, y_pred))
print("NMI: %.3f" % metrics.normalized_mutual_info_score(y, y_pred))
print("AMI: %.3f" % metrics.adjusted_mutual_info_score(y, y_pred))
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, y_pred))
print("Completeness: %0.3f" % metrics.completeness_score(y, y_pred))
print("V-measure: %0.3f" % metrics.v_measure_score(y, y_pred))
print("FMI: %.3f" % metrics.fowlkes_mallows_score(y, y_pred))

Adjusted Rand-Index: 0.051
NMI: 0.166
AMI: 0.158
Homogeneity: 0.161
Completeness: 0.171
V-measure: 0.166
FMI: 0.108


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [0]:
best_sil = float("-inf")
best_gamma = 1000
for my_gamma in gammas:
#  print("For gamma = ", my_gamma)
  km = KernelKMeans(n_clusters=20, max_iter=100, random_state=0, verbose=1, kernel="rbf", gamma=my_gamma)
  y_pred = km.fit_predict(tfidf_sub)
#  print("Computing silhouette_score ")
  silhouette = metrics.silhouette_score(tfidf_sub, y_pred)
  print("The silhouette_score for gamma = %f is = %.4f" % (my_gamma, silhouette))
  if silhouette > best_sil:
#    print("Old Sil : ", best_sil)
    best_sil = silhouette
    best_gamma = my_gamma
#    print("New Sil : ", best_sil, "gamma : ", best_gamma)

Converged at iteration 28
The silhouette_score for gamma = 0.010000 is = -0.0179
Converged at iteration 40
The silhouette_score for gamma = 0.100000 is = -0.2160
Converged at iteration 28
The silhouette_score for gamma = 1.000000 is = -0.2162
Converged at iteration 3
The silhouette_score for gamma = 10.000000 is = -0.2307
Converged at iteration 3
The silhouette_score for gamma = 100.000000 is = -0.2307


In [0]:
print("best_gamma : ", best_gamma)
km = KernelKMeans(n_clusters=20, max_iter=100, random_state=0, verbose=1, kernel="rbf", gamma=best_gamma)
y_pred = km.fit_predict(tfidf_sub)

best_gamma :  0.01
Converged at iteration 28


In [0]:
print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(y, y_pred))
print("NMI: %.3f" % metrics.normalized_mutual_info_score(y, y_pred))
print("AMI: %.3f" % metrics.adjusted_mutual_info_score(y, y_pred))
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, y_pred))
print("Completeness: %0.3f" % metrics.completeness_score(y, y_pred))
print("V-measure: %0.3f" % metrics.v_measure_score(y, y_pred))
print("FMI: %.3f" % metrics.fowlkes_mallows_score(y, y_pred))

Adjusted Rand-Index: 0.051
NMI: 0.166
AMI: 0.158
Homogeneity: 0.161
Completeness: 0.171
V-measure: 0.166
FMI: 0.108


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [0]:
best_sil = float("-inf")
best_gamma = 1000
for my_gamma in gammas:
#  print("For gamma = ", my_gamma)
  km = KernelKMeans(n_clusters=20, max_iter=100, random_state=0, verbose=1, kernel="rbf", gamma=my_gamma)
  y_pred = km.fit_predict(tfidf_bigram)
#  print("Computing silhouette_score ")
  silhouette = metrics.silhouette_score(tfidf_bigram, y_pred)
  print("The silhouette_score for gamma = %f is = %.4f" % (my_gamma, silhouette))
  if silhouette > best_sil:
#    print("Old Sil : ", best_sil)
    best_sil = silhouette
    best_gamma = my_gamma
#    print("New Sil : ", best_sil, "gamma : ", best_gamma)

Converged at iteration 38
The silhouette_score for gamma = 0.010000 is = -0.0185
Converged at iteration 32
The silhouette_score for gamma = 0.100000 is = -0.2202
Converged at iteration 39
The silhouette_score for gamma = 1.000000 is = -0.2207
Converged at iteration 3
The silhouette_score for gamma = 10.000000 is = -0.2340
Converged at iteration 3
The silhouette_score for gamma = 100.000000 is = -0.2340


In [0]:
print("best_gamma : ", best_gamma)
km = KernelKMeans(n_clusters=20, max_iter=100, random_state=0, verbose=1, kernel="rbf", gamma=best_gamma)
y_pred = km.fit_predict(tfidf_bigram)

best_gamma :  0.01
Converged at iteration 38


In [0]:
print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(y, y_pred))
print("NMI: %.3f" % metrics.normalized_mutual_info_score(y, y_pred))
print("AMI: %.3f" % metrics.adjusted_mutual_info_score(y, y_pred))
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, y_pred))
print("Completeness: %0.3f" % metrics.completeness_score(y, y_pred))
print("V-measure: %0.3f" % metrics.v_measure_score(y, y_pred))
print("FMI: %.3f" % metrics.fowlkes_mallows_score(y, y_pred))

Adjusted Rand-Index: 0.045
NMI: 0.160
AMI: 0.150
Homogeneity: 0.153
Completeness: 0.168
V-measure: 0.160
FMI: 0.107


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [0]:
best_sil = float("-inf")
best_gamma = 1000
for my_gamma in gammas:
#  print("For gamma = ", my_gamma)
  km = KernelKMeans(n_clusters=20, max_iter=100, random_state=0, verbose=1, kernel="rbf", gamma=my_gamma)
  y_pred = km.fit_predict(tfidf_max)
#  print("Computing silhouette_score ")
  silhouette = metrics.silhouette_score(tfidf_max, y_pred)
  print("The silhouette_score for gamma = %f is = %.4f" % (my_gamma, silhouette))
  if silhouette > best_sil:
#    print("Old Sil : ", best_sil)
    best_sil = silhouette
    best_gamma = my_gamma
#    print("New Sil : ", best_sil, "gamma : ", best_gamma)

Converged at iteration 33
The silhouette_score for gamma = 0.010000 is = -0.1041
Converged at iteration 38
The silhouette_score for gamma = 0.100000 is = -0.2083
Converged at iteration 11
The silhouette_score for gamma = 1.000000 is = -0.2509
Converged at iteration 3
The silhouette_score for gamma = 10.000000 is = -0.2574
Converged at iteration 3
The silhouette_score for gamma = 100.000000 is = -0.2574


In [0]:
print("best_gamma : ", best_gamma)
km = KernelKMeans(n_clusters=20, max_iter=100, random_state=0, verbose=1, kernel="rbf", gamma=best_gamma)
y_pred = km.fit_predict(tfidf_max)

best_gamma :  0.01
Converged at iteration 33


In [0]:
print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(y, y_pred))
print("NMI: %.3f" % metrics.normalized_mutual_info_score(y, y_pred))
print("AMI: %.3f" % metrics.adjusted_mutual_info_score(y, y_pred))
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, y_pred))
print("Completeness: %0.3f" % metrics.completeness_score(y, y_pred))
print("V-measure: %0.3f" % metrics.v_measure_score(y, y_pred))
print("FMI: %.3f" % metrics.fowlkes_mallows_score(y, y_pred))

Adjusted Rand-Index: 0.029
NMI: 0.142
AMI: 0.130
Homogeneity: 0.133
Completeness: 0.151
V-measure: 0.141
FMI: 0.099


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/cluster/supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
